# Library

In [1]:
! pip install wandb opencv-python-headless==4.1.2.30 albumentations torch-summary timm==0.5.4 einops joblib icecream  -qq -U

     |████████████████████████████████| 1.7 MB 13.2 MB/s 
     |████████████████████████████████| 21.8 MB 1.4 MB/s 
     |████████████████████████████████| 102 kB 63.0 MB/s 
     |████████████████████████████████| 431 kB 59.6 MB/s 
     |████████████████████████████████| 180 kB 59.6 MB/s 
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 142 kB 54.6 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 


In [2]:
from sklearn.metrics import f1_score
from glob import glob
import pathlib
from pathlib import Path
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pprint import pprint
import urllib.request
import csv
import numpy as np
from einops import rearrange, reduce, repeat
from torch.cuda import amp
from tqdm import tqdm
import wandb
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os
from icecream import ic
from sklearn.model_selection import train_test_split
import gc
import cv2
import copy
import time
import random
from PIL import Image

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

import timm

import json

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


from sklearn.metrics import f1_score

# SET SEED 

In [3]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed()


# Read the Data


In [18]:
import wandb
run = wandb.init(project="lg", entity="jiwon7258",
                 job_type='data')
                 
dataset = wandb.run.use_artifact(
    'jiwon7258/lg/lg:v1', type='dataset')

# # Download the artifact's contents
dataset_dir = dataset.download()
dataset_dir = Path(dataset_dir)


wandb: Currently logged in as: jiwon7258 (use `wandb login --relogin` to force relogin)


Remove jpg files

In [13]:
train_jpg = sorted(glob(str(TRAIN_PATH / '*/*.jpg')))
test_jpg = sorted(glob(str(TEST_PATH / '*/*.jpg')))


In [14]:
import os
for file_path in train_jpg :
  os.remove(file_path)
for file_path in test_jpg :
  os.remove(file_path)

In [19]:
artifact = wandb.Artifact('csv_json', type='dataset')
artifact.add_dir(dataset_dir)
# artifact.add_dir(TEST_PATH)

wandb: Adding directory to artifact (./artifacts/lg:v1)... Done. 16.1s


In [20]:
run.name = 'CSV_JSON'
run.log_artifact(artifact)

In [21]:
wandb.run.finish()